In [4]:
from skyfield.api import load, EarthSatellite, utc
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
from pytz import timezone  

In [5]:
name = "ISS (ZARYA)"
line1 = "1 25544U 98067A   24097.32390046  .00016717  00000+0  30203-3 0  9997"
line2 = "2 25544  51.6415 137.1456 0004685  86.2365  20.0504 15.50367909452557"
satellite = EarthSatellite(line1, line2, name)
ts = load.timescale()

In [6]:
utc = timezone('UTC')  
datetime_utc = utc.localize(datetime.now())  
t0 = ts.utc(datetime_utc)
times = ts.utc([datetime_utc + timedelta(minutes=i) for i in np.linspace(0, 90, 200)])

In [7]:
geocentric = satellite.at(times)
positions = geocentric.position.km  # [x, y, z] shape: (3, N)

In [8]:
# === Tracé Plotly ===
fig = go.Figure()

# Tracé de l'orbite
fig.add_trace(go.Scatter3d(
    x=positions[0], y=positions[1], z=positions[2],
    mode='lines',
    name='Orbital Path',
    line=dict(color='blue', width=2)
))

# Tracé de la Terre (approximation sphérique)
u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:100j]
earth_radius = 6371  # km
x = earth_radius * np.cos(u) * np.sin(v)
y = earth_radius * np.sin(u) * np.sin(v)
z = earth_radius * np.cos(v)

fig.add_trace(go.Surface(
    x=x, y=y, z=z,
    colorscale='Earth',
    opacity=0.6,
    showscale=False,
    name='Earth'
))

# Réglages de la visualisation
fig.update_layout(
    scene=dict(
        xaxis_title='X (km)', yaxis_title='Y (km)', zaxis_title='Z (km)',
        aspectmode='data'
    ),
    title=f'Orbite de {name}',
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=True
)

fig.show()
